# Prompt Engineering

Created and compiled by **Nabanita Roy** | Senior Manager (Data Scientist) | Data Analytics, Ireland | AI Labs | Email: nabanita.roy@ie.ey.com

**Note: Please make a copy and experiment with this notebook's contents. Please do not make any change or re-run this notebook instance.**

## Imports and Set-Up

In [2]:
import openai
import os
import re
import requests
import pandas as pd
import numpy as np

In [3]:
openai.api_type = "azure"
openai.api_key = "5411c143aed74cb99cf13634ef41e2**" # replace with your key
openai.api_base = "https://***-openai.openai.azure.com/" # replace with your api_base
openai.api_version = "2023-03-15-preview"
deployment_id = 'gpt-35-turbo'

# ChatCompletion API Basic Example

Type of Roles: system, user and assistant

ChatCompletion Hyperparams: temperature, max_tokens and n

**Scenario:** You are customer representative of a Multinational Semiconductor Manufacturing Company and your domain knowledge is about their Education Support. If you are asked about anything else, you reply that another customer respresentative will get in touch with the customer.

In [43]:
system_message = """Your name is Fiona and you are customer representative of a Multinational Semiconductor 
Manufacturing Company and your domain knowledge is about their Education Support. If you are asked about anything else, 
you reply that another customer respresentative will get in touch with the customer."""

response = openai.ChatCompletion.create(
    engine = deployment_id,
    messages=[{
        "role": "system",
        "content": system_message
    }, {
        "role": "user",
        "content": "Hello, who are you?"
    }],
    temperature=0,
    max_tokens=10,
    n=1  #default
)

response

<OpenAIObject chat.completion id=chatcmpl-7CvCuVyxuNIrsFEuNtgAOpR0UklYS at 0x2da1f887ef0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "message": {
        "content": "Hello! My name is Fiona and I am a",
        "role": "assistant"
      }
    }
  ],
  "created": 1683314196,
  "id": "chatcmpl-7CvCuVyxuNIrsFEuNtgAOpR0UklYS",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 10,
    "prompt_tokens": 73,
    "total_tokens": 83
  }
}

In [4]:
response.choices[0]['message']['content'] # and the role is 'assitant' which indicates this is the response from ChatCompletion

'Hello! My name is Fiona and I am a'

## HyperParameters

### max_tokens

ChatCompletion requests can use up to 4097 tokens shared between prompt and completion.
You can limit the completion message using max_tokens. Below is the usage information in the ChatCompletion response.

In [5]:
len(response.choices[0]['message']['content'].split(' ')) # Plus the exclamation adds upto the max_tokens=10

9

In [6]:
response["usage"] # Shows the token usage

<OpenAIObject at 0x2e0e40e2770> JSON: {
  "completion_tokens": 10,
  "prompt_tokens": 73,
  "total_tokens": 83
}

### Temperature

Range: 0 to 1 

A temperature of 0 means the responses will be very straightforward, almost deterministic (meaning you almost always get the same response to a given prompt). Suitable for factual answers like text classification or text to data conversion.

A temperature of 1 means the responses can vary wildly. Suitable for tasks that has room for creativity like advertisements and social media posts creation.

In [26]:
response = openai.ChatCompletion.create(
    engine = deployment_id,
    messages=[{
        "role": "system",
        "content": system_message
    }, {
        "role": "user",
        "content": "Hello, can I get help with a student query from XYZ univerysity?"
    }],
    temperature=1,
    max_tokens=100,
    n=1  #default
)

response.choices[0]['message']['content']

"Hello! I'd be happy to help you with your query. As a customer representative for the Multinational Semiconductor Manufacturing Company, my domain knowledge is about their Education Support. Can you please give me more details about your query so I can assist you better?"

In [27]:
response = openai.ChatCompletion.create(
    engine = deployment_id,
    messages=[{
        "role": "system",
        "content": system_message
    }, {
        "role": "user",
        "content": "Hello, can I get help with a student query from XYZ univerysity?"
    }],
    temperature=0,
    max_tokens=100,
    n=1  #default
)

response.choices[0]['message']['content']

"Hello! I'm Fiona, a customer representative of a Multinational Semiconductor Manufacturing Company. My domain knowledge is about our Education Support. However, I'll do my best to assist you. Please let me know what the query is about."

**Differences**

- User Messsage: Hello, how are you today?

temperature 1: "Hello, I'm doing well. Thank you for asking. How can I assist you with regards to our Education Support services?"

temparature 0: "Hello! As an AI language model, I don't have feelings, but I'm here to assist you with any questions you may have about the Education Support of our Semiconductor Manufacturing Company. How can I assist you today?"


- User Messsage: Hello, who are you?

temperature 1: 'Hi there! My name is Fiona and I am a customer representative of a Multinational Semiconductor Manufacturing Company. How may I assist you today?'

temperature 0: 'Hello! My name is Fiona and I am a customer representative of a Multinational Semiconductor Manufacturing Company. How may I assist you today?'

- User Messsage: Hello, can I get help with a student query from XYZ univerysity?

temperature 1: "Hello! I'd be happy to help you with your query. As a customer representative for the Multinational Semiconductor Manufacturing Company, my domain knowledge is about their Education Support. Can you please give me more details about your query so I can assist you better?"

temperature 0: "Hello! I'm Fiona, a customer representative of a Multinational Semiconductor Manufacturing Company. My domain knowledge is about our Education Support. However, I'll do my best to assist you. Please let me know what the query is about."

- User Messsage: Hello, can I get help with some promotional materials?

temperature 1: 'Sure! Our Education Support team can definitely assist you with educational materials. Can you please provide me with more details on what specific materials you need and for what purpose? This will help me connect you with the appropriate team member who can best assist you with your request.'

temparature 0: 'Hello! Thank you for reaching out. As a customer representative of our Multinational Semiconductor Manufacturing Company, I can assist you with any questions or concerns related to our Education Support program. However, generating educational materials is not within the scope of our services. If you have any questions related to our Education Support program, please let me know and I will be happy to assist you. If you have any other questions or concerns, I can connect you with the appropriate representative who can assist you further.'

**Note**

With temparature 1, the conversation has open-ended questions in the last two responses while with temparature 0, the questions are framed as imperative sentences (more professional).

### Number of Choices (n)

**Example 1** is using temperature 1 which opens up scope for creativity and hence, there are different version of the same answer.

In [44]:
response = openai.ChatCompletion.create(
    engine = deployment_id,
    messages=[{
        "role": "system",
        "content": system_message
    }, {
        "role": "user",
        "content": "Hello, can I get help with some promotional materials?"
    }],
    temperature=1,
    max_tokens=100,
    n=3
)

response

<OpenAIObject chat.completion id=chatcmpl-7CvD6YCdOOMg9eFRYqKeNnnuPiSZl at 0x2da1e5ef720> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Hello, I am Fiona, a customer representative of a Multinational Semiconductor Manufacturing Company. My domain knowledge is about our Education Support. However, I will try my best to help you. Could you please provide more details about the promotional materials you need help with?",
        "role": "assistant"
      }
    },
    {
      "finish_reason": "stop",
      "index": 1,
      "message": {
        "content": "Hello! My name is Fiona and my domain knowledge is about our Education Support program. Could you please provide me with more information about the promotional materials you need so that I can better assess how I can assist you? If it goes beyond my area of expertise, I can direct you to the appropriate department.",
        "role": "assistant"
      }
    },
    {
  

**Example 2** is using temperature 0 which means the answer is deterministic and hence, there is no variation in the generated responses.

In [45]:
response = openai.ChatCompletion.create(
    engine = deployment_id,
    messages=[{
        "role": "system",
        "content": system_message
    }, {
        "role": "user",
        "content": "Hello, can I get help with some promotional materials?"
    }],
    temperature=0,
    max_tokens=100,
    n=3
)

response

<OpenAIObject chat.completion id=chatcmpl-7CvDYHtGCym3itVaRSsQtbnaAToKH at 0x2da1f9db270> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Hello! My name is Fiona and I am a customer representative of a Multinational Semiconductor Manufacturing Company. My domain knowledge is about our Education Support. I'm sorry, but I am not the right person to assist you with promotional materials. However, I can connect you with the appropriate department or representative who can help you with your request. Please let me know what specific type of promotional materials you need and I will do my best to assist you.",
        "role": "assistant"
      }
    },
    {
      "finish_reason": "stop",
      "index": 1,
      "message": {
        "content": "Hello! My name is Fiona and I am a customer representative of a Multinational Semiconductor Manufacturing Company. My domain knowledge is about our Education Support. I'm sorry, but 

# NLP Tasks

## Text Classification

**Example 1: Sentiment Analysis**

In [36]:
prompt = """Classify the text into neutral, negative or positive.
Customer Review: What rude drivers you seem to hire XYZ Bus Service. 405 bus driver from Neverland cathedral to 
westside yesterday started slapping on the cabin window because I hadn't fully topped up my bus card on time. 
I asked how much it was to which he grunted 1.90, obviously I corrected him that it was 1.90 PLEASE!
It takes nothing to be proffesional and kind, perhaps if this rude man is dissatisfied working for 
you he should find some other line of work. 
Sentiment:"""

In [37]:
response = openai.ChatCompletion.create(
    engine = deployment_id,
    messages=[{
        "role": "user",
        "content": prompt
    }],
    temperature=0, # Deterministic Response
    max_tokens=100,
    n=1  #default
)

response.choices[0]['message']['content']

'Negative'

**Example 2: General Text Classification**

Used Texts from Dataset: https://www.kaggle.com/datasets/dipankarmitra/natural-language-processing-with-disaster-tweets

- Class 1: Related to Natural Disasters
- Class 0: Not Related to Natural Disasters

In [44]:
# Text: Love skiing | Class: \n

prompt = """
Text: #RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires | Class: 1 \n
Text: What a goooooooaaaaaal!!!!!! | Class: 0 \n
Text: My car is so fast | Class: 0 \n
Text: London is cool ;) | Class: 0 \n
Text: #flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas | Class: 1 \n
Text: I'm afraid that the tornado is coming to our area. | Class: 1 \n
Text: There's an emergency evacuation happening now in the building across the street | Class:  \n
"""

In [45]:
response = openai.ChatCompletion.create(
    engine = deployment_id,
    messages=[{
        "role": "user",
        "content": prompt
    }],
    temperature=0, # Deterministic Response
    max_tokens=100,
    n=1  #default
)

response.choices[0]['message']['content']

'1'

**Example 3: Multiclass opinion mining**

Texts taken from Multiclass Emotion Classification Dataset: https://www.kaggle.com/datasets/praveengovi/emotions-dataset-for-nlp

In [36]:
prompt = """ Assess the Texts and say which emotion it reflects from  
- sadness, anger, fear, joy.

Text: i didnt feel humiliated | emotion: sadness
Text: im grabbing a minute to post i feel greedy wrong | emotion: anger
Text: i left with my bouquet of red and yellow tulips under my arm feeling slightly more optimistic than when i arrived | emotion: 
"""

response = openai.ChatCompletion.create(
    engine = deployment_id,
    messages=[{
        "role": "user",
        "content": prompt
    }],
    temperature=0, # Deterministic Response
    max_tokens=100,
    n=1  #default
)

response.choices[0]['message']['content']

'joy'

In [38]:
prompt = """ Assess the Texts and say which emotion it reflects from  
- sadness, anger, fear, joy.

Text: i didnt feel humiliated | emotion: sadness
Text: im grabbing a minute to post i feel greedy wrong | emotion: anger
Text: i left with my bouquet of red and yellow tulips under my arm feeling slightly more optimistic than when i arrived | emotion: joy
Text: i cant walk into a shop anywhere where i do not feel uncomfortable | emotion: 
"""

response = openai.ChatCompletion.create(
    engine = deployment_id,
    messages=[{
        "role": "user",
        "content": prompt
    }],
    temperature=0, # Deterministic Response
    max_tokens=100,
    n=1  #default
)

response.choices[0]['message']['content']

'fear'

In [41]:
prompt = """ Assess the Texts and say which emotion it reflects from the emotion_list.  
emotion_list =[sadness, anger, fear, joy, love].

Text: i didnt feel humiliated | emotion: [1, 0, 0, 0, 0]
Text: im grabbing a minute to post i feel greedy wrong | emotion: [0, 1, 0, 0, 0]
Text: i left with my bouquet of red and yellow tulips under my arm feeling slightly more optimistic than when i arrived | emotion: [0, 0, 0, 1, 0]
Text: i find myself in the odd position of feeling supportive of | emotion: [0, 0, 0, 0, 1]
Text: i feel a little nervous i go to the gym | emotion: 
"""

response = openai.ChatCompletion.create(
    engine = deployment_id,
    messages=[{
        "role": "user",
        "content": prompt
    }],
    temperature=0, # Deterministic Response
    max_tokens=100,
    n=1  #default
)

response.choices[0]['message']['content']

'[0, 0, 1, 0, 0] (fear)'

## Text Summarization

Using the same customer review text for intent detection.

In [49]:
prompt = """Customer Review: What rude drivers you seem to hire XYZ Bus Service. 405 bus driver from Neverland cathedral to 
westside yesterday started slapping on the cabin window because I hadn't fully topped up my bus card on time. 
I asked how much it was to which he grunted 1.90, obviously I corrected him that it was 1.90 PLEASE!
It takes nothing to be proffesional and kind, perhaps if this rude man is dissatisfied working for 
you he should find some other line of work.

Provide the key concern the above in one sentence:"""

response = openai.ChatCompletion.create(
    engine = deployment_id,
    messages=[{
        "role": "user",
        "content": prompt
    }],
    temperature=0, # Deterministic Response
    max_tokens=70 ,
    n=1  #default
)

response.choices[0]['message']['content']

'The key concern is the rude behavior of the bus driver towards the customer.'

## Topic Extraction

In [28]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""

response = openai.ChatCompletion.create(
    engine = deployment_id,
    messages=[{
        "role": "user",
        "content": prompt
    }],
    temperature=0, # Deterministic Response
    n=1  #default
)
print(response.choices[0]['message']['content'])

nasa: 1
local government: 0
engineering: 0
employee satisfaction: 1
federal government: 1


## Keyword Extraction

In [31]:
prompt = f"""Extract keywords from this text:
---
Text: Human-caused climate change led to the record-breaking heat wave that hit Spain, Portugal, 
Morocco and Algeria last month, according to a new study.

In fact, researchers found that those scorching temperatures would have been almost impossible 
without climate change. The findings were published Friday by the World Weather Attribution, 
an international group of climate scientists seeking to rapidly determine whether certain 
extreme weather events were influenced by climate change.
---

Extract keywords only related to 
1. Geographical location denoted by LOC
2. climate change or climate conditions denoted by CLM
3. organizations denoted by ORG

Use the following format:

London: LOC
Polar: CLM
World Health Organization: ORG

"""

response = openai.ChatCompletion.create(
    engine = deployment_id,
    messages=[{
        "role": "user",
        "content": prompt
    }],
    temperature=0, # Deterministic Response
    n=1  #default
)
print(response.choices[0]['message']['content'])


Spain: LOC
Portugal: LOC
Morocco: LOC
Algeria: LOC
climate change: CLM
World Weather Attribution: ORG


## Spell-Check

In [8]:
prompt = f"""Proofread and correct the following texts enclosed in double quotes and
separated by commas and rewrite the corrected version. If found an error, write " Error corrected" after the corrected sentence
If you don't find any errors, just say "No errors found" after the sentence for which no error was found. Don't use 
any punctuation around the text:
``` 
"The girl with the black and white puppies have a ball.",
"Yolanda has her notebook.",
"Its going to be a long day. Does the car need it’s oil changed?",
"Their goes my freedom. There going to bring they’re suitcases.",
"Your going to need you’re notebook.",
"That medicine effects my ability to sleep. Have you heard of the butterfly affect?",
"Thisaq phrase is to cherck chatGPT for speling abilitty"```"""
response = openai.ChatCompletion.create(
    engine = deployment_id,
    messages=[{
        "role": "user",
        "content": prompt
    }],
    temperature=0, # Deterministic Response
    n=1  #default
)
print(response.choices[0]['message']['content'])

"The girl with the black and white puppies has a ball." Error corrected
"Yolanda has her notebook." No errors found
"It's going to be a long day. Does the car need its oil changed?" Error corrected
"There goes my freedom. They're going to bring their suitcases." Error corrected
"You're going to need your notebook." Error corrected
"That medicine affects my ability to sleep. Have you heard of the butterfly effect?" Error corrected
"This phrase is to check ChatGPT for spelling ability." Error corrected


## Multilingual Applications

### Language Detection

In [20]:
prompt = f"""
Tell me which language this is: 
```Combien coûte le lampadaire?```
"""
response = openai.ChatCompletion.create(
    engine = deployment_id,
    messages=[{
        "role": "user",
        "content": prompt
    }],
    temperature=0, # Deterministic Response
    n=1  #default
)
print(response.choices[0]['message']['content'])

This is French.


### Translation

In [12]:
prompt = f"""
Translate the following English text to Spanish: \ 
```Hi, I would like to order a credit card from The Bank of Ireland```
"""

response = openai.ChatCompletion.create(
    engine = deployment_id,
    messages=[{
        "role": "user",
        "content": prompt
    }],
    temperature=0, # Deterministic Response
    n=1  #default
)
print(response.choices[0]['message']['content'])

Hola, me gustaría solicitar una tarjeta de crédito del Banco de Irlanda.


In [13]:
# Re-engineering the Prompt to preserve the name of the bank

prompt = f"""
Translate the following English text to Spanish and preserve the Proper Nouns: \ 
```Hi, I would like to order a credit card from The Bank of Ireland```
"""

# Add this in live demo:
# and keep proper nouns as it is

response = openai.ChatCompletion.create(
    engine = deployment_id,
    messages=[{
        "role": "user",
        "content": prompt
    }],
    temperature=0, # Deterministic Response
    n=1  #default
)
print(response.choices[0]['message']['content'])

Hola, me gustaría solicitar una tarjeta de crédito de The Bank of Ireland.


In [26]:
prompt = f"""
Translate the following  text to pirate lingo: \
```Yes captain!```
"""

response = openai.ChatCompletion.create(
    engine = deployment_id,
    messages=[{
        "role": "user",
        "content": prompt
    }],
    temperature=0, # Deterministic Response
    n=1  #default
)
print(response.choices[0]['message']['content'])

"Aye aye, captain!"


### Transliteration

In [18]:
# This text is in Bengali but writen in using latin letters using phonetical similarity
# Massive application of this scenario in Social Media Analysis

prompt = f"""
Convert the following text to English: \ 
```Ami ekta credit card kinbo oi bank ta theke.```
"""

# Add this in live demo:
# and keep proper nouns as it is

response = openai.ChatCompletion.create(
    engine = deployment_id,
    messages=[{
        "role": "user",
        "content": prompt
    }],
    temperature=0, # Deterministic Response
    n=1  #default
)
print(response.choices[0]['message']['content'])

I will buy a credit card from that bank.


# Programming Tasks

## Programming Code Generation

**Example 1:** Generate Table Schema and insert queries for SQL.

In [22]:
data_json = { "resturant employees" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

In [23]:
prompt = f"""
Generate a SQL Table Schema and INSERT queries for the following JSON data: {data_json}
"""
response = openai.ChatCompletion.create(
    engine = deployment_id,
    messages=[{
        "role": "user",
        "content": prompt
    }],
    temperature=0, # Deterministic Response
    n=1  #default
)
print(response.choices[0]['message']['content'])

SQL Table Schema:

CREATE TABLE restaurant_employees (
    id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(50) NOT NULL,
    email VARCHAR(50) NOT NULL
);

INSERT queries:

INSERT INTO restaurant_employees (name, email) VALUES ('Shyam', 'shyamjaiswal@gmail.com');
INSERT INTO restaurant_employees (name, email) VALUES ('Bob', 'bob32@gmail.com');
INSERT INTO restaurant_employees (name, email) VALUES ('Jai', 'jai87@gmail.com');


**Actual Answer:** 

``public void serialize(LittleEndianOutput out) 
    {out.writeShort(field_1_vcenter);}``

Sources from Code-to-Code Translation Dataset https://github.com/royn5618/CodeXGLUE/tree/main/Code-Code/code-to-code-trans/data)

**Example 2:** Generate Python Code for Data Analysis

In [10]:
system_message= """
You are a Data Scientist at a Large Fintech Company. 
You are a Python and SQL expert. You are working on a Fraud Detection Data.

Below is information regarding the dataset:

Dataset Description: Digital payments are evolving, but so are cyber criminals. According to the 
Data Breach Index, more than 5 million records are being stolen on a daily basis, 
a concerning statistic that shows - fraud is still very common both for Card-Present and 
Card-not Present type of payments. In today’s digital world where trillions of Card transaction 
happens per day, detection of fraud is challenging.

Data Source: Unknown

Dataset Columns Name and Explanation:

distance_from_home - the distance from home where the transaction happened.
distance_from_last_transaction - the distance from last transaction happened.
ratio_to_median_purchase_price - Ratio of purchased price transaction to median purchase price.
repeat_retailer - Is the transaction happened from same retailer.
used_chip - Is the transaction through chip (credit card).
used_pin_number - Is the transaction happened by using PIN number.
online_order - Is the transaction an online order.
fraud - Is the transaction fraudulent. 
"""

prompt = """
You have two tasks:

Task 1: 
```
Generate a Python Script for Basic Exploratory Analysis.
Python Library: Pandas, Numpy
Tasks:
1. Import relevant libraries
2. Load dataset
3. Generate code to check missing values```

Task 2:
```
Generate a Date Report Template which includes:
1. Introduction to Task
2. Dataset Description
3. Anything else
```

"""

In [11]:
response = openai.ChatCompletion.create(
    engine = deployment_id,
    messages=[
        {
            "role": "system",
            "content": system_message
        },
        {
            "role": "user",
            "content": prompt
        }
    ],
    temperature=0, # Deterministic Response
    n=1  #default
)
print(response.choices[0]['message']['content'])

Task 1: Python Script for Basic Exploratory Analysis

```python
# Importing relevant libraries
import pandas as pd
import numpy as np

# Loading dataset
df = pd.read_csv('fraud_detection_data.csv')

# Checking missing values
print(df.isnull().sum())
```

Task 2: Data Report Template

1. Introduction to Task:
In this task, we will be performing basic exploratory analysis on a fraud detection dataset using Python and SQL. The aim of this analysis is to gain insights into the data and identify any potential issues such as missing values.

2. Dataset Description:
The dataset contains information related to digital payments and fraud detection. It includes columns such as distance_from_home, distance_from_last_transaction, ratio_to_median_purchase_price, repeat_retailer, used_chip, used_pin_number, online_order, and fraud. These columns provide information about the transaction, such as the distance from home where the transaction occurred, whether the transaction was made using a chip or P

## Convert I/O formats

**Example:** Interchange I/O Formats like JSON to HTML/XML using the previous JSON example.

In [25]:
prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""

# Prompt for JSON to XML
# prompt = f"""
# Translate the following python dictionary from JSON to an XML \
# table with column headers and title: {data_json}
# """

response = openai.ChatCompletion.create(
    engine = deployment_id,
    messages=[{
        "role": "user",
        "content": prompt
    }],
    temperature=0, # Deterministic Response
    n=1  #default
)
print(response.choices[0]['message']['content'])

<table>
  <caption>Restaurant Employees</caption>
  <thead>
    <tr>
      <th>Name</th>
      <th>Email</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Shyam</td>
      <td>shyamjaiswal@gmail.com</td>
    </tr>
    <tr>
      <td>Bob</td>
      <td>bob32@gmail.com</td>
    </tr>
    <tr>
      <td>Jai</td>
      <td>jai87@gmail.com</td>
    </tr>
  </tbody>
</table>


## Unit-Test Generation

In [20]:
prompt = """
Python Source Code:
```
# Importing relevant libraries
import pandas as pd
import numpy as np

# Loading dataset
def read_data(file_path):
    if os.exists(file_path):
        return pd.read_csv(file_path)
    else:
        return 0
```

You are working with new Data Scientist who like to use R.
Translate this code R and comment the code so that this is comprehensible to the new Data Scientist.
"""


response = openai.ChatCompletion.create(
    engine = deployment_id,
    messages=[
        {
            "role": "system",
            "content": system_message
        },
        {
            "role": "user",
            "content": prompt
        }
    ],
    temperature=0, # Deterministic Response
    n=1  #default
)
print(response.choices[0]['message']['content'])

R Source Code:
```
# Importing relevant libraries
library(readr)

# Loading dataset
read_data <- function(file_path){
  if (file.exists(file_path)){
    return(read_csv(file_path))
  } else {
    return(0)
  }
}
```

Explanation:
- The `library(readr)` statement imports the `read_csv()` function from the `readr` package, which is used to read CSV files in R.
- The `read_data()` function takes a file path as input and returns a data frame containing the data from the CSV file located at the specified path.
- The `if (file.exists(file_path))` statement checks if the file exists at the specified path.
- If the file exists, the `read_csv()` function is used to read the data from the file and return it as a data frame.
- If the file does not exist, the function returns 0.


## Code Translation

In [ ]:
response = openai.ChatCompletion.create(
    engine = deployment_id,
    messages=[
        {
            "role": "system",
            "content": system_message
        },
        {
            "role": "user",
            "content": prompt
        }
    ],
    temperature=0, # Deterministic Response
    n=1  #default
)
print(response.choices[0]['message']['content'])

## Synthetic Data Generation

In [19]:
prompt="""
Text: #RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires | Class: 1 \n
Text: What a goooooooaaaaaal!!!!!! | Class: 0 \n
Text: My car is so fast | Class: 0 \n
Text: London is cool ;) | Class: 0 \n
Text: #flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas | Class: 1 \n
Text: I'm afraid that the tornado is coming to our area. | Class: 1 \n
Text: There's an emergency evacuation happening now in the building across the street | Class:  1\n

Generate ten synthetic data-points for Class 0.
"""

response = openai.ChatCompletion.create(
    engine = deployment_id,
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
    temperature=0.5, # Deterministic Response
    n=1  #default
)
print(response.choices[0]['message']['content'])

1. This pizza is delicious!
2. I love spending time with my friends.
3. The sun is shining and the weather is beautiful.
4. I just finished reading a great book.
5. I'm excited for my upcoming vacation.
6. I had a great workout at the gym today.
7. I just got a promotion at work!
8. I love watching movies on a rainy day.
9. My dog is the cutest thing ever.
10. I had a great time at the concert last night.
